# Web service

Since Parselmouth is a normal Python library, it can also easily be used within the context of a web server. There are several Python frameworks that allow to quickly set up a web server or web service. In this examples, we will use [Flask](https://flask.palletsprojects.com/) to show how easily one can set up a web service that uses Parselmouth to access Praat functionality such as the pitch track estimation algorithms. This functionality can then be accessed by clients without requiring either Praat, Parselmouth, or even Python to be installed, for example within the context of an online experiment.

All that is needed to set up the most basic web server in Flask is a single file. We adapt [the standard Flask example](https://flask.palletsprojects.com/quickstart/) to accept a sound file, access Parselmouth's [Sound.to_pitch](../api_reference.rst#parselmouth.Sound.to_pitch), and then send back the list of pitch track frequencies. Note that apart from [saving the file that was sent in the HTTP request](https://flask.palletsprojects.com/quickstart/#file-uploads) and encoding the resulting list of frequencies in JSON, the Python code of the `pitch_track` function is the same as one would write in a normal Python script using Parselmouth.

In [ ]:
%%writefile server.py

from flask import Flask, request, jsonify
import tempfile

app = Flask(__name__)

@app.route('/pitch_track', methods=['POST'])
def pitch_track():
    import parselmouth

    # Save the file that was sent, and read it into a parselmouth.Sound
    with tempfile.NamedTemporaryFile() as tmp:
        tmp.write(request.files['audio'].read())
        sound = parselmouth.Sound(tmp.name)
    
    # Calculate the pitch track with Parselmouth
    pitch_track = sound.to_pitch().selected_array['frequency']
    
    # Convert the NumPy array into a list, then encode as JSON to send back
    return jsonify(list(pitch_track))

Normally, we can then run the server typing `FLASK_APP=server.py flask run` on the command line, as explained in the [Flask documentation](https://flask.palletsprojects.com/quickstart/). Please do note that to run this server publicly, in a secure way and as part of a bigger setup, other options are available to deploy! Refer to the [Flask deployment documentation](https://flask.palletsprojects.com/deploying/).

However, to run the server from this Jupyter notebook and still be able to run the other cells that access the functionality on the client side, the following code will start the server in a separate thread and print the output of the running server.

In [ ]:
import os
import subprocess
import sys
import time

# Start a subprocess that runs the Flask server
p = subprocess.Popen([sys.executable, "-m", "flask", "run"], env=dict(**os.environ, FLASK_APP="server.py"), stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Start two subthreads that forward the output from the Flask server to the output of the Jupyter notebook
def forward(i, o):
    while p.poll() is None:
        l = i.readline().decode('utf-8')
        if l:
            o.write("[SERVER] " + l)

import threading
threading.Thread(target=forward, args=(p.stdout, sys.stdout)).start()
threading.Thread(target=forward, args=(p.stderr, sys.stderr)).start()

# Let's give the server a bit of time to make sure it has started
time.sleep(2)

Now that the server is up and running, we can make a standard HTTP request to this web service. For example, we can send a Wave file with an audio recording of someone saying *"The north wind and the sun [...]"*: [the_north_wind_and_the_sun.wav](audio/the_north_wind_and_the_sun.wav), extracted from a [Wikipedia Commons audio file](https://commons.wikimedia.org/wiki/File:Recording_of_speaker_of_British_English_%28Received_Pronunciation%29.ogg).

In [ ]:
from IPython.display import Audio
Audio(filename="audio/the_north_wind_and_the_sun.wav")

To do so, we use the [requests library](https://requests.readthedocs.io/) in this example, but we could use any library to send a standard HTTP request.

In [ ]:
import requests
import json

# Load the file to send
files = {'audio': open("audio/the_north_wind_and_the_sun.wav", 'rb')}
# Send the HTTP request and get the reply
reply = requests.post("http://127.0.0.1:5000/pitch_track", files=files)
# Extract the text from the reply and decode the JSON into a list
pitch_track = json.loads(reply.text)
print(pitch_track)

Since we used the standard `json` library from Python to decode the reply from server, `pitch_track` is now a normal list of `float`s and we can for example plot the estimated pitch track:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.set() # Use seaborn's default style to make attractive graphs
plt.rcParams['figure.dpi'] = 100 # Show nicely large images in this notebook

In [ ]:
plt.figure()
plt.plot([float('nan') if x == 0.0 else x for x in pitch_track], '.')
plt.show()

Refer to the [examples on plotting](plotting.rst) for more details on using Parselmouth for plotting.

Importantly, Parselmouth is thus only needed by the server; the client only needs to be able to send a request and read the reply. Consequently, we could even use a different programming language on the client's side. For example, one could make build a HTML page with JavaScript to make the request and do something with the reply:

```html
<head>
    <meta http-equiv="content-type" content="text/html; charset=UTF-8" />
    <script type="text/javascript" src="jquery.min.js"></script>
    <script type="text/javascript" src="plotly.min.js"></script>
    <script type="text/javascript">
    var update_plot = function() {
        var audio = document.getElementById("audio").files[0];
        var formData = new FormData();
        formData.append("audio", audio);

        $.getJSON({url: "http://127.0.0.1:5000/pitch_track", method: "POST",
                   data: formData, processData: false, contentType: false,
                   success: function(data){
                       Plotly.newPlot("plot", [{ x: [...Array(data.length).keys()],
                                                 y: data.map(function(x) { return x == 0.0 ? undefined : x; }),
                                                 type: "lines" }]);}});
    };
    </script>
</head>
<body>
<form onsubmit="update_plot(); return false;">
    <input type="file" name="audio" id="audio" />
    <input type="submit" value="Get pitch track" />
    <div id="plot" style="width:1000px;height:600px;"></div>
</form>
</body>
```

Again, one thing to take into account is the security of running such a web server. However, apart from deploying the flask server in a secure and performant way, we also need one extra thing to circumvent a standard security feature of the browser. Without handling Cross Origin Resource Sharing (CORS) on the server, the JavaScript code on the client side will not be able to access the web service's reply. A Flask extension exists however, [Flask-CORS](https://flask-cors.readthedocs.io/en/latest/), and we refer to its documentation for further details.

In [ ]:
# Let's shut down the server
p.kill()

In [ ]:
# Cleaning up the file that was written to disk
!rm server.py